### 0. Import Packages

In [1]:
import time
import serial
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

### 1. Declare COM ports for Serial Communication (RS-232)

* **Balance:** Mettler Toledo MR3002
* **Pump**: Ismatec Reglo ICC 4-channel, 8-roller peristaltic pump

In [2]:
balance_port = 'COM8'
pump_port = 'COM9'

### 2. Balance Control

In [3]:
def get_mass(port):
    try:
        balance = serial.Serial(port, 9600, xonxoff=True) # check COM port and xonxoff (handshake) = True default for MT balance
        balance.write('SI\r\n'.encode('utf-8')) # sent immediate weight vs. 'S\r\n' send stable weight only
        time.sleep(0.4) # sleep for 0.4 seconds - time between writing and reading from balance
        result  = balance.read_until('\r\n'.encode('utf-8')) # read until <CR><LF>, i.e., full response
        value = str(result[7:14].decode("utf-8")) # strip numerical result from raw output
        balance.close()
        if (value.strip() != '-------') and (value.strip() != ''):
            return (True,float(value)) # return a status T vs. F, in addition to the mass reading - useful for future steps (data logging)
        return (False,-1)
    except:
        return (False,-1)

### 3. Pump Control

In [4]:
from PeriPump import RegloICC
pump = RegloICC(pump_port) # initialise pump 

The available functions from the pump class are `start_channel`, `stop_channel`, `set_direction`, `get_direction`, `set_speed`, `get_speed`, `set_mode` ,`get_mode`.

In [5]:
# Example pump commands

# ### Set the rotation direction of channel 3 to clockwise
# pump.set_direction(3, 0)

# ### Get the rotation direction of channel 3
# print(pump.get_direction(3))

# ### Set the operational mode of channel 3 to RPM
# pump.set_mode(3, 0)

# ### Get the current operational mode of channel 3
# print(pump.get_mode(3))


# ### Get the current speed setting of channel 3
# print(pump.get_speed(3))

### Set the independent channel speeds
# pump.set_speed(1, 50)
# pump.set_speed(2, 100) # max speed 100 RPM
# pump.set_speed(3, 100) # max speed 100 RPM
# pump.set_speed(4, 50)

# pump.start_channel(1)
# pump.start_channel(2)
# pump.start_channel(3)
# pump.start_channel(4)

# ### Stop channel 3
# pump.stop_channel(4)

# ### Delete the pump object
# del pump

### 4. AutoDose Programme

In [6]:
# Map ingredients to pump channels

pump_dict = {'SLES70':1,
            'CAPB': 2, 
            'Water': 3,
            'SLES25':4}

In [8]:
def dose_ingredient(ing, amt, balance_port, pump_speed=100):
    '''
    ing: ingredient name (should match in pump_dict)
    amt: mass to be dosed (g)
    balance_port: serial connection to mass balance (COM #)
    pump_speed: set in RPM NB. 100 RPM = max, set at max, opt. argument
    '''
    t = [0]  # time 
    m = [get_mass(balance_port)[1]] # mass - initialise with initial mass
    # NB. get_mass returns tuple (STATUS, MASS), therefore, [1] to access mass value.

    i = 0 # initialise counter

    channel = pump_dict[ing]
    pump.set_speed(channel, pump_speed) # max speed = 100 RPM

    while m[-1] < m[0] + amt:

        # Turn on and keep the pump going till the target amount is reached.
        
        pump.start_channel(channel)

        status, val = get_mass(balance_port)
        if status:
            curr_val = val
        else:
            curr_val = m[-1] 
        m.append(curr_val)

        i += 1
        t.append(i*0.5) # 0.4 s sleep time between reading & writing to balance + 0.1 s execution lag
        # NB. Therefore, mass is recorded every 0.5 seconds.

    pump.stop_channel(channel)

    disp_m = m[-1] - m[0]
    acc = np.round((np.absolute(disp_m - amt)/amt)*100,2) # accuracy

    return np.round(disp_m,2), acc, m, t

#### Single dosing

In [9]:
disp_m, acc, m, t = dose_ingredient('Water', 10, balance_port)
print(disp_m, acc)

10.08 0.8


#### Multiple dosing

In [ ]:
# sles_results = [] 

# water_doses = [2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
# sles_doses = [3, 6, 10]

# for i in sles_doses:
#     result = dose_ingredient('SLES25', i, balance_port)
#     sles_results.append(result)

### 5. Results & Analysis

In [ ]:
# water_results = results

# water_accuracy = []
# water_flowrate = []

# doses = len(water_results)

# for i in range(doses):
#     water_accuracy.append(float(water_results[i][1]))
#     time = water_results[i][3][-1] - water_results[i][3][0] # s
#     mass = water_results[i][2][-1] - water_results[i][2][0] # g 
#     water_flowrate.append((mass/time)*60) # g/min

# print(np.round(np.mean(water_accuracy),2))
# print(np.round(np.mean(water_flowrate),2))

0.44
22.03


In [ ]:
# sles_accuracy = []
# sles_flowrate = []

# doses = len(sles_results)

# for i in range(doses):
#     sles_accuracy.append(float(sles_results[i][1]))
#     time = sles_results[i][3][-1] - sles_results[i][3][0] # s
#     mass = sles_results[i][2][-1] - sles_results[i][2][0] # g 
#     sles_flowrate.append((mass/time)*60) # g/min

# print(np.round(np.mean(sles_accuracy),2))
# print(np.round(np.mean(sles_flowrate),2))

1.55
16.75


In [12]:
# plt.boxplot(water_accuracy);
# plt.title('AutoDose - 2 - 150 g');
# plt.xticks([1], ['Water']);
# plt.ylabel('Dosing error (%)');


In [ ]:
# plt.plot(water_results[-1][3], water_results[-1][2]);
# plt.xlabel('Time/s');
# plt.ylabel('Mass/g');
# plt.title('AutoDose - Water');

#### Limitations

* Speed/time to dispense (viscous) ingredients.
* Cleaning of lines - recommend not changing lines on core viscous materials or replacing tubing as required.
* Independent channels allows for accuracy (vs. closed-system, e.g., UoS) but speed is comprimised and we cannot operate the channels simultaneously because then we would not be able to measure the individual masses of ingredients added.